In [1650]:
# importing packages
import math
import numpy as np
import pandas as pd

In [1651]:
# helper function for getting probability from frequency table
# this function is used in get_entropy()
def get_probability(event_info):
  SUM = sum(event_info)
  for i in range(len(event_info)):
        event_info[i] /= SUM
  return event_info


# this function gets entropy from frequency table
def get_entropy(event_info):
  probabilities = get_probability(event_info)
  # print('probabilities', probabilities)
  entropy = 0.0
  for p in probabilities:
    if p != 0:
      entropy += p * math.log(1 / p) / math.log(2)
  return entropy

# this function gets gini impurity from frequency table
def get_gini_impurity(event_info):
  probabilities = get_probability(event_info)
  # print('probabilities', probabilities)
  imp = 0.0
  for p in probabilities:
    if p != 0:
      imp += p * (1.0 - p)
  return imp


In [1652]:
#this funtion sort table by column
def sort_table_by_column(table, col):
    return table.sort_values(by = [col]).reset_index(drop=True)

In [1653]:
# for importing the dataset as a numpy array
dataset1 = pd.read_csv(r"covid_dataset.csv")
dataset1 = dataset1[dataset1['location'] == 'India']
# dataset1 = dataset1.to_numpy()
dataset2 = pd.read_csv(r"changes-visitors-covid.csv")
dataset2 = dataset2[dataset2['Entity'] == 'India']
# dataset2 = dataset2.to_numpy()
dataset3 = pd.read_csv(r'continuous_attribute_table.csv')
dataset4 = pd.read_csv(r'pure_category.csv')

In [1654]:
dataset = pd.merge(dataset1, dataset2, on = 'date', how = 'inner')

In [1655]:
example_dataset = dataset1 = pd.read_csv(r"example_data.csv")

In [1656]:
print(example_dataset)

  Outlook  Value Company Sailboat target
0   sunny    100     big    small    yes
1   sunny    200     med    small    yes
2   sunny    300     med      big    yes
3   sunny    400      no    small    yes
4   sunny    500     big      big    yes
5   rainy     10      no    small     no
6   rainy    500     med    small    yes
7   rainy    510     big      big    yes
8   rainy     12      no      big     no
9   rainy      9     med      big     no


In [1657]:
def split_dataset_wrt_column(dataset, column_name):
  unique_items = dataset[column_name].unique()
  tables = []
  for item in unique_items:
    tables.append(dataset[dataset[column_name] == item])
  return tables

In [1658]:
def get_count(table, target_column, class_name):
  # print(class_name)
  # ans =  (table[target_column] == class_name).shape[0]
  ans = (table[target_column] == class_name).sum()
  # print (ans)
  return ans

In [1659]:
def get_entropy_from_table(table, target_column):
  unique_classes = table[target_column].unique()
  # print(unique_classes)
  counts = []
  for class_name in unique_classes:
    counts.append(get_count(table,target_column, class_name))
  # print("Count is ", counts)
  return get_entropy(counts)    # write get_gini_impurity if you want to change the parameter to gini imp from entropy
  

In [1660]:
# this function returns the information gain of the column "column" when the target column is "target_column" of the table dataset
# only for categorical column or attribute
def get_information_gain(dataset, column, target_column):
  tables = []
  size_table = []
  overall_size = dataset.shape[0]
  for table in split_dataset_wrt_column(dataset, column):
    tables.append(table)
    size_table.append(table.shape[0])
  entropies = []
  for table in tables:
    # print(table)
    entropies.append(get_entropy_from_table(table, target_column))
  # print("entropies=", entropies)
  # entropies = [get_entropy_from_table(table, target_column) for table in tables]
  # print(entropies)
  entropy_initial = get_entropy_from_table(dataset, target_column)    # entropy without splitting
  # print("entropy_intial=",entropy_initial)
  entropy = sum([(size / overall_size) * entropyi for size, entropyi in zip(size_table, entropies)])  # entropy after splitting
  return (entropy_initial - entropy)


In [1661]:
get_information_gain(example_dataset, 'Sailboat', 'target')

0.034851554559677034

In [1662]:
def max_index(arr):
    index = 0
    mx = arr[0]
    for i in range(len(arr)):
        if mx < arr[i]:
            index = i
            mx = arr[i]
    return index


In [1663]:
get_information_gain(example_dataset, 'Sailboat', 'target')

0.034851554559677034

In [1664]:
def get_value_with_min_entropy_wrt_continuous_column(table, column, target_column):
    # step 1: sort the table
    new_table = sort_table_by_column(table, column)
    # print(new_table)
    # step 2: get various averages
    avg_array = []
    length_new_table = len(new_table)
    for i in range(length_new_table - 1):
        avg_array.append((new_table.at[i,column] + new_table.at[i + 1, column]) / 2)
    
    # print(avg_array)
    # step 3: count before and after averages
    IGs = []
    parentIG = get_entropy_from_table(new_table, target_column)
    for i in range(length_new_table - 1):
        table1 = new_table.iloc[:i + 1,:]
        table2 = new_table.iloc[i + 1:, :]
        # print('Table 1')
        # print(table1)
        # print('Table 2')
        # print(table2)
        E1 = get_entropy_from_table(table1, target_column)
        E2 = get_entropy_from_table(table2, target_column)
        # print('E1=',E1, 'E2=', E2)
        E = (len(table1) / len(table)) * E1 + (len(table2) / len(table)) * E2
        IG = parentIG - E
        IGs.append(IG)
    # print(IGs)
    if(len(avg_array) == 0):
        IGs = [0]
        avg_array = [0]
    index = max_index(IGs)
    
    return avg_array[index], IGs[index]  # split wrt value, IG according to that value
    # step 4: calculate the entropy wrt each average
    # step 5: determine the best split with most information gain
    

In [1665]:
get_value_with_min_entropy_wrt_continuous_column(dataset3, 'Weight', 'Heart_disease')

(205.0, 0.41997309402197514)

In [1666]:
dataset3

,Weight,Heart_disease
0,155,No
1,190,No
2,220,Yes
3,225,Yes
4,180,Yes


In [1667]:
#This function returns the best column for the split
def get_best_column (table, target_column, is_categorical):
    values = []
    # is_categorical is an array which is true if the data is categorical and false if continuous
    IGs = []
    for index,column in enumerate(table):
        if(column == target_column):
            break
        # print(column,index)
        if(is_categorical[index] == 1):
            values.append(None)
            IGs.append(get_information_gain(table, column, target_column))
        else:
            #value is the value at which the splitting occurs in the column and IG is the corresponding Info gain
            value, IG = get_value_with_min_entropy_wrt_continuous_column(table,column,target_column)
            values.append(value)
            # print(value)
            IGs.append(IG)
    selected_index = max_index(IGs)
    if(is_categorical[selected_index]):
        return selected_index, table.columns[selected_index], None, IGs[selected_index]
    else:
        return selected_index, table.columns[selected_index],values[selected_index], IGs[selected_index]



In [1668]:
get_best_column(example_dataset, 'target', [1,0,1,1])

(1, 'Value', 56.0, 0.8812908992306927)

In [1669]:
def split_table_wrt_value(table,value,column):
    table1 = table[table[column] <= value]
    table2 = table[table[column] > value]
    return table1,table2

In [1670]:
table1, table2 = split_table_wrt_value(example_dataset,56.0,'Value')
print(table1)
print()
print(table2)

  Outlook  Value Company Sailboat target
5   rainy     10      no    small     no
8   rainy     12      no      big     no
9   rainy      9     med      big     no

  Outlook  Value Company Sailboat target
0   sunny    100     big    small    yes
1   sunny    200     med    small    yes
2   sunny    300     med      big    yes
3   sunny    400      no    small    yes
4   sunny    500     big      big    yes
6   rainy    500     med    small    yes
7   rainy    510     big      big    yes


In [1671]:
class Node:
    def __init__(self, condition, children, decisions):
        self.condition = condition
        self.children = children
        self.decisions = decisions

In [1672]:
class Leaf:
    def __init__(self, leaf_value):
        self.leaf_value = leaf_value
        self.condition = "THIS IS A LEAF NODE!!"
        

In [1673]:
# this cell contains all the constants please beware
THRESHOLD_VALUE = 0.0
IS_CATEGORICAL = [1, 0, 0, 0, 0, 0, 0, 0]
TARGET_COLUMN = "new_cases_classes"

In [1674]:
# returns the value and probability for the leaf which have maximum probability
def get_value_for_leaf(table, target_column):
    freq = dict()
    unique_classes = table[target_column].unique()
    for class_name in unique_classes:
        freq[class_name] = 0
    for index, row in table.iterrows():
        freq[row[target_column]] += 1
    mx = 0
    value = None
    sum_of_freq = 0.0
    for class_name, freq_of_class in freq.items():
        sum_of_freq += freq_of_class
        if mx < freq_of_class:
            mx = freq_of_class
            value = class_name
    return value, mx / sum_of_freq

In [1675]:
def build_tree(table):
    # step1 find the best split
    selected_index, column_name, value, best_IG = get_best_column(table, TARGET_COLUMN, IS_CATEGORICAL)
    tables = []
    decisions = []
    condition = None
    # categorical value
    if value == None:
        tables = split_dataset_wrt_column(table, column_name)
        start_index = 0
        for splited_table in tables:
            decisions.append(splited_table.iloc[0][column_name])
            start_index += len(splited_table)
        condition = column_name
    # Continuous Value
    else:
        table1, table2 = split_table_wrt_value(table, value, column_name)
        tables.append(table1)
        tables.append(table2)
        decisions = [None,None]
        condition = [value, column_name]
    # put some base condition
    if best_IG <= THRESHOLD_VALUE:
        return Leaf(get_value_for_leaf(table, TARGET_COLUMN))
    # make tree for each child
    children = []
    for table in tables:
        child = build_tree(table)
        children.append(child)
    # return the current node which is already linked to its children so that current node's parent can link current node
    return Node(condition, children,decisions)

In [1676]:
# root = build_tree(example_dataset) # before running this cell change the IS_CATEGORICAL array

In [1677]:
# print(root.condition)
# for child in root.children:
#     print(child.condition)
#     if (child.condition == "THIS IS A LEAF NODE!!"):
#         print(child.leaf_value)

In [1678]:
# root = build_tree(dataset4)

In [1679]:
# print(root.condition)
# for child in root.children:
#     print(child.condition)
#     if (child.condition == "THIS IS A LEAF NODE!!"):
#         print(child.leaf_value)

In [1680]:

def print_tree(root, spacing=""):
    if isinstance(root, Leaf):
        print(spacing, root.leaf_value)
        return
    print(spacing, root.condition,root.decisions)
    for child in root.children:
        print_tree(child,spacing+"--> ")

    
    

In [1681]:
# print_tree(root)

In [1682]:
def find_index(arr,x):
    for i in range (len(arr)):
        if(arr[i] == x):
            return i

In [1683]:
def predict(row,root):
    #base case
    if(isinstance(root,Leaf)):
        return root.leaf_value
    value_to_check = row.at[0, root.condition]
    child_index = find_index(root.decisions, value_to_check)
    return predict(row, root.children[child_index])

        


In [1684]:
# test_data = pd.DataFrame({
#     'Outlook': ['rainy'],
#     'Company': ['big'],
#     'Sailboat': ['small']
# })

In [1685]:
# print(predict(test_data, root))

In [1686]:
dataset5 = pd.read_csv(r'processed_covid_data.csv')[300:320]

In [1687]:
dataset5

,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,new_cases_classes
300,-28.714,7.571,10.000,-12.000,-17.286,-19.857,46
301,-29.143,7.286,10.571,-11.571,-17.571,-19.286,47
302,-28.143,9.286,10.571,-9.286,-16.714,-18.143,47
303,-28.429,9.143,11.143,-9.286,-17.143,-17.571,47
304,-28.714,9.000,11.571,-9.714,-17.714,-17.000,48
305,-28.714,9.143,11.714,-9.857,-18.000,-16.429,48
306,-28.571,9.286,11.714,-9.571,-18.000,-15.286,48
307,-28.429,9.429,11.714,-9.429,-17.857,-15.000,48
308,-28.143,9.857,11.714,-9.143,-17.571,-14.571,49
309,-28.143,9.571,11.714,-9.286,-17.714,-14.571,49


In [1688]:
root = build_tree(dataset5)

In [1689]:
print_tree(root)

 retail_and_recreation [-28.714, -29.143, -28.143, -28.429, -28.571, -27.857, -27.286, -26.429, -26.571, -26.714, -25.714, -25.857]
-->  [8.285499999999999, 'grocery_and_pharmacy'] [None, None]
--> -->  (46, 1.0)
--> -->  (48, 1.0)
-->  (47, 1.0)
-->  [9.4285, 'grocery_and_pharmacy'] [None, None]
--> -->  (47, 1.0)
--> -->  (49, 1.0)
-->  [9.286000000000001, 'grocery_and_pharmacy'] [None, None]
--> -->  (47, 1.0)
--> -->  (48, 1.0)
-->  (48, 1.0)
-->  (49, 1.0)
-->  (50, 1.0)
-->  (50, 1.0)
-->  [-7.9285, 'transit_stations'] [None, None]
--> -->  [10.1425, 'grocery_and_pharmacy'] [None, None]
--> --> -->  (51, 1.0)
--> --> -->  (50, 1.0)
--> -->  (52, 1.0)
-->  (50, 1.0)
-->  (52, 1.0)
-->  (52, 1.0)
